In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv("./2021Census_G01_VIC_SA1.csv")
total_pop = df["Tot_P_P"]
total_pop




0         435
1         184
2         377
3         584
4         358
         ... 
15477     493
15478       0
15479      21
15480      14
15481    9089
Name: Tot_P_P, Length: 15482, dtype: int64

In [3]:
# 1. Parents (Age 25 - 55) born in China 

df = pd.read_csv("./2021Census_G09F_VIC_SA1.csv")

china_born_df = df.loc[:, "P_China_25_44":"P_China_45_54"].copy()

SA1s = df.iloc[:, 0]

china_born_df.insert(0, "SA1 Code", SA1s)

china_born_df["Chinese Parents (25-55)"] = china_born_df["P_China_25_44"] + china_born_df["P_China_45_54"]

china_born_df = china_born_df.drop(china_born_df.columns[[1, 2]], axis=1)

china_born_df.sort_values("Chinese Parents (25-55)", ascending=False).head(20)

china_born_df.insert(2, "Total Pop", total_pop)
china_born_df["Prop_CN_Parents"] = china_born_df["Chinese Parents (25-55)"]/china_born_df["Total Pop"]
china_born_df.sort_values("Prop_CN_Parents", ascending=False)

china_born_df

,SA1 Code,Chinese Parents (25-55),Total Pop,Prop_CN_Parents
0,20101100101,0,435,0.000000
1,20101100102,0,184,0.000000
2,20101100105,0,377,0.000000
3,20101100106,0,584,0.000000
4,20101100107,0,358,0.000000
...,...,...,...,...
15477,21704148035,4,493,0.008114
15478,29797979991,0,0,NaN
15479,29797979992,0,21,0.000000
15480,29797979993,0,14,0.000000


In [4]:
# Taiwan + Singapore

df = pd.read_csv("./2021Census_G09H_VIC_SA1.csv")

tw_sing_df = df[["SA1_CODE_2021","P_Taiwan_25_44", "P_Taiwan_45_54", "P_Singapore_25_44", "P_Singapore_45_54"]].copy()

tw_sing_df["target_parents"] = tw_sing_df["P_Taiwan_25_44"] + tw_sing_df["P_Taiwan_45_54"] + tw_sing_df["P_Singapore_25_44"] + tw_sing_df["P_Singapore_45_54"]

tw_sing_df = tw_sing_df.iloc[:, [0, 5]]
tw_sing_df



,SA1_CODE_2021,target_parents
0,20101100101,0
1,20101100102,0
2,20101100105,0
3,20101100106,0
4,20101100107,0
...,...,...
15477,21704148035,0
15478,29797979991,0
15479,29797979992,0
15480,29797979993,0


In [5]:
# India

df = pd.read_csv("./2021Census_G09G_VIC_SA1.csv")

ind_df = df[["SA1_CODE_2021", "P_India_25_44", "P_India_45_54"]].copy()

ind_df["Indian Parents"] = ind_df["P_India_25_44"] + ind_df["P_India_45_54"]

ind_df = ind_df.iloc[:, [0,3]]
ind_df

,SA1_CODE_2021,Indian Parents
0,20101100101,0
1,20101100102,0
2,20101100105,7
3,20101100106,0
4,20101100107,4
...,...,...
15477,21704148035,4
15478,29797979991,0
15479,29797979992,0
15480,29797979993,0


In [6]:
# Join tw_sing and ind

tw_sing_ind_df = pd.merge(tw_sing_df, ind_df, on="SA1_CODE_2021", how="outer").copy()

tw_sing_ind_df["combined target parents"] = tw_sing_ind_df["target_parents"] + tw_sing_ind_df["Indian Parents"]

tw_sing_ind_df = tw_sing_ind_df.iloc[:, [0,3]]

tw_sing_ind_df

,SA1_CODE_2021,combined target parents
0,20101100101,0
1,20101100102,0
2,20101100105,7
3,20101100106,0
4,20101100107,4
...,...,...
15477,21704148035,4
15478,29797979991,0
15479,29797979992,0
15480,29797979993,0


In [7]:
# Join rest with china

asian_df = pd.merge(tw_sing_ind_df, china_born_df, left_on="SA1_CODE_2021", right_on="SA1 Code", how="outer")
asian_df = asian_df[["SA1 Code", "combined target parents", "Chinese Parents (25-55)", "Total Pop"]].copy()
asian_df["asian parents"] = asian_df["combined target parents"] + asian_df["Chinese Parents (25-55)"]
asian_df = asian_df.iloc[:, [0, 4, 3]]
asian_df = asian_df.sort_values("asian parents", ascending=False)
asian_df

,SA1 Code,asian parents,Total Pop
3513,20604150504,599,2967
13459,21305158607,365,1100
13376,21305158312,339,915
10793,21204131011,285,1157
4984,20703116334,285,909
...,...,...,...
2705,20505110003,0,591
2706,20505110004,0,633
2707,20505110005,0,0
2708,20505110006,0,349


In [20]:
# 2. Children with Parents born in China 
df = pd.read_csv("./2021Census_G08_VIC_SA1.csv")
chinese_parents_df = df[["SA1_CODE_2021", "Chinese_Tot_resp"]].copy()
chinese_parents_df.rename(columns={'Chinese_Tot_resp': 'Children w/ Chinese Parents'}, inplace=True)
chinese_parents_df.sort_values("Children w/ Chinese Parents", ascending=False)

chinese_parents_df.insert(2, "Total Pop", total_pop)
chinese_parents_df["Prop_Ch_w_CN_Parents"] = chinese_parents_df["Children w/ Chinese Parents"]/chinese_parents_df["Total Pop"]
chinese_parents_df.sort_values("Prop_Ch_w_CN_Parents", ascending=False)

chinese_parents_df[chinese_parents_df["SA1_CODE_2021"] == 20703116357]
# df[["SA1_CODE_2021", "Chinese_Tot_resp", "Taiwanese_Tot_resp"]]
print(df.keys().tolist())

['SA1_CODE_2021', 'Aust_BP_B_OS', 'Aust_FO_B_OS', 'Aust_MO_B_OS', 'Aust_Both_parents_born_Aust', 'Aust_Birthplace_not_stated', 'Aust_Tot_resp', 'Aust_Abor_BP_B_OS', 'Aust_Abor_FO_B_OS', 'Aust_Abor_MO_B_OS', 'Aust_Abor_BP_B_Aus', 'Aust_Abor_BP_NS', 'Aust_Abor_Tot_resp', 'Chinese_BP_B_OS', 'Chinese_FO_B_OS', 'Chinese_MO_B_OS', 'Chinese_BP_B_Aus', 'Chinese_BP_NS', 'Chinese_Tot_resp', 'Croatian_BP_B_OS', 'Croatian_FO_B_OS', 'Croatian_MO_B_OS', 'Croatian_BP_B_Aus', 'Croatian_BP_NS', 'Croatian_Tot_resp', 'Dutch_BP_B_OS', 'Dutch_FO_B_OS', 'Dutch_MO_B_OS', 'Dutch_BP_B_Aus', 'Dutch_BP_NS', 'Dutch_Tot_resp', 'English_BP_B_OS', 'English_FO_B_OS', 'English_MO_B_OS', 'English_BP_B_Aus', 'English_BP_NS', 'English_Tot_resp', 'Filipino_BP_B_OS', 'Filipino_FO_B_OS', 'Filipino_MO_B_OS', 'Filipino_BP_B_Aus', 'Filipino_BP_NS', 'Filipino_Tot_resp', 'French_BP_B_OS', 'French_FO_B_OS', 'French_MO_B_OS', 'French_BP_B_Aus', 'French_BP_NS', 'French_Tot_resp', 'German_BP_B_OS', 'German_FO_B_OS', 'German_MO_B_OS'

In [9]:
# 3. Children currently attending Primary School
# 4. Children currently attending Secondary School 
df = pd.read_csv("./2021Census_G15_VIC_SA1.csv")
students_df = df[["SA1_CODE_2021", "Primary_Tot_Primary_P", "Secondary_Tot_Secondary_P"]].copy()
students_df.rename(columns={"Primary_Tot_Primary_P": "Primary"}, inplace=True)
students_df.rename(columns={"Secondary_Tot_Secondary_P": "Secondary"}, inplace=True)
students_df.sort_values("Secondary", ascending=False)

students_df.insert(3, "Total Pop", total_pop)
students_df["Prop_Primary"] = students_df["Primary"]/students_df["Total Pop"]
students_df["Prop_Secondary"] = students_df["Secondary"]/students_df["Total Pop"]

# students_df.sort_values("Prop_Primary", ascending=False)
students_df.sort_values("Prop_Secondary", ascending=False)

# students_df

,SA1_CODE_2021,Primary,Secondary,Total Pop,Prop_Primary,Prop_Secondary
8104,21005125019,0,3,4,0.000000,0.750000
12410,21303135111,0,4,6,0.000000,0.666667
8390,21101125407,0,4,6,0.000000,0.666667
1247,20302148603,20,347,543,0.036832,0.639042
142,20101100825,22,179,367,0.059946,0.487738
...,...,...,...,...,...,...
15365,21704147808,0,0,0,NaN,NaN
15387,21704147902,0,0,0,NaN,NaN
15436,21704147954,0,0,0,NaN,NaN
15455,21704148013,0,0,0,NaN,NaN


In [10]:
# 5. Families with total reported income of >$100,000 
Sheet_A = pd.read_csv("./2021Census_G58A_VIC_SA1.csv")
Sheet_B = pd.read_csv("./2021Census_G58B_VIC_SA1.csv")
income_df = Sheet_A[["SA1_CODE_2021", "I_1_149_Tot", "I_150_299_Tot", "I_300_399_Tot", "I_400_499_Tot", "I_500_649_Tot", "I_650_799_Tot", "I_800_999_Tot", "I_1000_1249_Tot", "I_1250_1499_Tot", "I_1500_1749_Tot", "I_1750_1999_Tot", "I_2000_2499_Tot", "I_2500_2999_Tot", "I_3000_3499_Tot", "I_3500_3999_Tot"]].copy()

high_income = Sheet_B[["I_4000more_Tot"]].copy()
income_df = income_df.merge(high_income, left_index=True, right_index=True)
income_df["High Income"] = income_df[["I_2000_2499_Tot", "I_2500_2999_Tot", "I_3000_3499_Tot", "I_3500_3999_Tot","I_4000more_Tot"]].sum(axis=1)
income_df["Total"] = income_df[["I_1_149_Tot", "I_150_299_Tot", "I_300_399_Tot", "I_400_499_Tot", "I_500_649_Tot", "I_650_799_Tot", "I_800_999_Tot", "I_1000_1249_Tot", "I_1250_1499_Tot", "I_1500_1749_Tot", "I_1750_1999_Tot", "I_2000_2499_Tot", "I_2500_2999_Tot", "I_3000_3499_Tot", "I_3500_3999_Tot", "I_4000more_Tot"]].sum(axis=1)
income_df["Prop_Aff"] = income_df["High Income"] / income_df["Total"]
income_df = income_df[["SA1_CODE_2021", "High Income", "Total", "Prop_Aff"]]


income_df





 

,SA1_CODE_2021,High Income,Total,Prop_Aff
0,20101100101,30,39,0.769231
1,20101100102,11,11,1.000000
2,20101100105,8,14,0.571429
3,20101100106,37,57,0.649123
4,20101100107,3,3,1.000000
...,...,...,...,...
15477,21704148035,43,55,0.781818
15478,29797979991,0,0,NaN
15479,29797979992,0,0,NaN
15480,29797979993,0,0,NaN


In [11]:
# Concatenate all the DataFrames in the `dfs` list into a single DataFrame
combined_df = pd.concat([china_born_df, chinese_parents_df.drop(["SA1_CODE_2021", "Total Pop"], axis=1), students_df.drop(["SA1_CODE_2021", "Total Pop"], axis=1), income_df.drop("SA1_CODE_2021", axis=1)], axis=1)

# Drop SA1s with <50 people because not efficient + high chance of inaccurate data
combined_df = combined_df.drop(combined_df[combined_df['Total Pop'] < 50].index)

# first normalize the proportions to be on the same scale
combined_df['Prop_CN_Parents_z'] = combined_df['Prop_CN_Parents'] / combined_df['Prop_CN_Parents'].max()
combined_df['Prop_Ch_w_CN_Parents_z'] = combined_df['Prop_Ch_w_CN_Parents'] / combined_df['Prop_Ch_w_CN_Parents'].max()
combined_df['Prop_Secondary_z'] = combined_df['Prop_Secondary'] / combined_df['Prop_Secondary'].max()
combined_df['Prop_Primary_z'] = combined_df['Prop_Primary'] / combined_df['Prop_Primary'].max()
combined_df['Prop_Aff_z'] = combined_df['Prop_Aff'] / combined_df['Prop_Aff'].max()

# create a score with weights
combined_df['Score'] = (combined_df['Prop_CN_Parents_z'] * 0.55 +
               combined_df['Prop_Ch_w_CN_Parents_z'] * 0.05 +
               combined_df['Prop_Secondary_z'] * 0.35 +
               combined_df['Prop_Primary_z'] * 0.1 +
               combined_df['Prop_Aff_z'] * 0.05)

# # # made a list of all valid SA3 regions
# sa3_region_df = pd.read_csv("./SA3 Region - Sheet1.csv")
# valid_SA3 = sa3_region_df['SA3s within 20km Radius'].tolist()

# # filtered combined_df to only keep rows which have SA3 values in the valid SA3 list
# valid_regions = combined_df[combined_df['SA1 Code'].astype(str).str.startswith(tuple(str(num) for num in valid_SA3))]
# valid_regions

# # made a list of all valid SA2 regions
sa2_region_df = pd.read_csv("./SA2s - Sheet1.csv")
valid_SA2 = sa2_region_df["SA2 Code"].tolist()
valid_SA2

# filtered combined_df to only keep rows which have SA2 values in the valid SA2 list
valid_regions = combined_df[combined_df['SA1 Code'].astype(str).str.startswith(tuple(str(num) for num in valid_SA2))]
valid_regions

# sort by score and select top 20
top_regions = valid_regions.sort_values('Score', ascending=False).head(10)
top_regions


,SA1 Code,Chinese Parents (25-55),Total Pop,Prop_CN_Parents,Children w/ Chinese Parents,Prop_Ch_w_CN_Parents,Primary,Secondary,Prop_Primary,Prop_Secondary,High Income,Total,Prop_Aff,Prop_CN_Parents_z,Prop_Ch_w_CN_Parents_z,Prop_Secondary_z,Prop_Primary_z,Prop_Aff_z,Score
8420,21101125438,92,578,0.159170,309,0.534602,65,41,0.112457,0.070934,96,123,0.780488,0.458098,0.639415,0.111001,0.479378,0.780488,0.409737
9194,21104127224,49,353,0.138810,118,0.334278,40,37,0.113314,0.104816,48,66,0.727273,0.399503,0.399815,0.164020,0.483034,0.727273,0.381791
9183,21104127213,73,619,0.117932,199,0.321486,79,69,0.127625,0.111470,79,106,0.745283,0.339414,0.384516,0.174433,0.544037,0.745283,0.358623
9088,21104126909,50,349,0.143266,143,0.409742,30,16,0.085960,0.045845,25,32,0.781250,0.412328,0.490075,0.071741,0.366428,0.781250,0.352099
11514,21205132704,53,454,0.116740,182,0.400881,44,44,0.096916,0.096916,51,60,0.850000,0.335984,0.479477,0.151659,0.413132,0.850000,0.345659
9213,21104127318,34,278,0.122302,98,0.352518,25,19,0.089928,0.068345,19,19,1.000000,0.351992,0.421632,0.106950,0.383343,1.000000,0.340444
8561,21101125937,33,301,0.109635,94,0.312292,41,27,0.136213,0.089701,32,41,0.780488,0.315534,0.373520,0.140368,0.580644,0.780488,0.338437
11543,21205132734,31,309,0.100324,83,0.268608,27,45,0.087379,0.145631,40,47,0.851064,0.288736,0.321271,0.227890,0.372475,0.851064,0.334431
9225,21104127330,41,355,0.115493,125,0.352113,31,31,0.087324,0.087324,41,51,0.803922,0.332394,0.421147,0.136648,0.372242,0.803922,0.329121
11518,21205132708,44,385,0.114286,145,0.376623,34,35,0.088312,0.090909,37,51,0.725490,0.328920,0.450463,0.142258,0.376453,0.725490,0.327139


In [12]:
[top_regions["Total"].sum(), top_regions["Total Pop"].sum()]

[596, 3981]

In [13]:
top_regions.describe()

,SA1 Code,Chinese Parents (25-55),Total Pop,Prop_CN_Parents,Children w/ Chinese Parents,Prop_Ch_w_CN_Parents,Primary,Secondary,Prop_Primary,Prop_Secondary,High Income,Total,Prop_Aff,Prop_CN_Parents_z,Prop_Ch_w_CN_Parents_z,Prop_Secondary_z,Prop_Primary_z,Prop_Aff_z,Score
count,1.000000e+01,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,10.00000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,2.113383e+10,50.000000,398.100000,0.123796,149.600000,0.366314,41.600000,36.400000,0.102543,0.091189,46.80000,59.60000,0.804526,0.356290,0.438133,0.142697,0.437117,0.804526,0.351748
std,4.921908e+07,19.223828,116.690902,0.017805,67.294048,0.072572,17.475062,15.108497,0.018580,0.027078,23.85046,32.10469,0.081558,0.051243,0.086801,0.042373,0.079203,0.081558,0.025975
min,2.110113e+10,31.000000,278.000000,0.100324,83.000000,0.268608,25.000000,16.000000,0.085960,0.045845,19.00000,19.00000,0.725490,0.288736,0.321271,0.071741,0.366428,0.725490,0.327139
25%,2.110413e+10,35.750000,319.000000,0.114588,103.000000,0.324684,30.250000,28.000000,0.087612,0.075032,33.25000,42.50000,0.754084,0.329788,0.388341,0.117413,0.373470,0.754084,0.335432
50%,2.110413e+10,46.500000,354.000000,0.117336,134.000000,0.352315,37.000000,36.000000,0.093422,0.090305,40.50000,51.00000,0.780869,0.337699,0.421390,0.141313,0.398238,0.780869,0.343051
75%,2.117988e+10,52.250000,436.750000,0.134683,172.750000,0.394817,43.250000,43.250000,0.113100,0.102841,50.25000,64.50000,0.838480,0.387625,0.472224,0.160930,0.482120,0.838480,0.356992
max,2.120513e+10,92.000000,619.000000,0.159170,309.000000,0.534602,79.000000,69.000000,0.136213,0.145631,96.00000,123.00000,1.000000,0.458098,0.639415,0.227890,0.580644,1.000000,0.409737
